In [ ]:
M = 5 #[m for m in range(1,20)]
J = 3
K = 3
d_mj = 50 #[d for d in range(50,100)]


Fm = 1.0*1e9
Fj = 5.0*1e9
f0 = 5.0*1e9


E_m = 3.6

S_m = 0.8*(1e-25)
S_j = 1.05*(1e-27)

I_mk = 0.8

alpha_mk = 75

beta_1,beta_2 = 1,0.2
beta_3,beta_4 = 1,0.2

W_uj,W_dj = 15,15

h1,h2 = 0.98,0.98

G_0 = 1.6 * (1e-11)

v = 4

P_Fjt = 0.25
P_mr = P_m0 = 0.5
P_mt = 0.1
P_theta1 = P_theta2 = 0.001

rho = 18

r0 = 1

e0 = 1

T_mstar = 4

A_1j = 2e10
A_2j = 4e10
G_1j = 2*1e3
G_2j = 4*1e3

X_mk = 75


E_m = 4.5
E_j = 15
E_0 = 30

N = 1e10

In [ ]:
[np.random.uniform(0,1) for i in range(100)]

In [ ]:
def energy(po):
#     print(po)
    def cal_Em():
        e = 0.0
        for m in range(M):
            for k in range(K):
                lambda_mk1 = po[3][m][k]
                f_mk = po[1][m][k]
                e += S_m * I_mk * lambda_mk1 * alpha_mk * f_mk ** 2
        return e

    def cal_Rmj(m, j):
        mu_mj = po[-1][m][j]
        p_mt = po[-2][m]
        R_mju = mu_mj * W_uj * np.log(1 + (p_mt * np.float_power(d_mj, -v) * (h1 ** 2)) / G_0)
        R_mjd = mu_mj * W_dj * np.log(1 + (P_Fjt * np.float_power(d_mj, -v) * (h2 ** 2)) / G_0)
        return R_mju, R_mjd

    def cal_Fai_mj(m, j):
        fai_mj1 = 0.0
        fai_mj2 = 0.0

        for k in range(K):
            mu_mj = po[-1][m][j]
            p_mt = po[-2][m]
            lambda_mk2 = po[4][m][k]
            lambda_mk3 = po[5][m][k]
            R_mju, R_mjd = cal_Rmj(m, j)
            uplink_fai = (P_m0 + rho * mu_mj * p_mt) * (beta_1 * (lambda_mk2 + lambda_mk3) * I_mk) / R_mju
            downlink_fai = (P_mr * beta_2 * (lambda_mk2 + lambda_mk3) * I_mk) / R_mjd

            fai_mj1 += (uplink_fai + downlink_fai)

            f_mjk = po[2][m][j][k]
            fai_mj2 += S_j * I_mk * lambda_mk2 * alpha_mk * (f_mjk ** 2)

        return fai_mj1 + fai_mj2

    def cal_Emj():

        def cal_fai_m0(m):
            fai_m0 = 0.0
            for k in range(K):
                lambda_mk3 = po[5][m][k]
                uplink_fai = (P_theta1 * beta_3 * lambda_mk3 * I_mk) / r0
                downlinke_fai = (P_theta2 * beta_4 * lambda_mk3 * I_mk) / r0
                execute_fai = I_mk * lambda_mk3 * alpha_mk * e0

                fai_m0 += (uplink_fai + downlinke_fai + execute_fai)

            return fai_m0

        total = 0.0
        for m in range(M):
            for j in range(J):
                x_mj = po[0][m][j]
                fai_mj = cal_Fai_mj(m, j)
                fai_m0 = cal_fai_m0(m)
                E_mj = (fai_mj + fai_m0)
                total += x_mj * E_mj

        return total

    optim = cal_Em() + cal_Emj()
#     print(cal_Em(),cal_Emj())
    return optim


def optim_score(po):
    def cal_Em():
        e = 0.0
        for m in range(M):
            for k in range(K):
                lambda_mk1 = po[3][m][k]
                f_mk = po[1][m][k]
                e += S_m * I_mk * lambda_mk1 * alpha_mk * f_mk ** 2
        return e

    def cal_Rmj(m, j):
        mu_mj = po[-1][m][j]
        p_mt = po[-2][m]
        R_mju = mu_mj * W_uj * np.log(1 + (p_mt * np.float_power(d_mj, -v) * (h1 ** 2)) / G_0)
        R_mjd = mu_mj * W_dj * np.log(1 + (P_Fjt * np.float_power(d_mj, -v) * (h2 ** 2)) / G_0)
        return R_mju, R_mjd

    def cal_Fai_mj(m, j):
        fai_mj1 = 0.0
        fai_mj2 = 0.0

        for k in range(K):
            mu_mj = po[-1][m][j]
            p_mt = po[-2][m]
            lambda_mk2 = po[4][m][k]
            lambda_mk3 = po[5][m][k]
            R_mju, R_mjd = cal_Rmj(m, j)
            uplink_fai = (P_m0 + rho * mu_mj * p_mt) * (beta_1 * (lambda_mk2 + lambda_mk3) * I_mk) / R_mju
            downlink_fai = (P_mr * beta_2 * (lambda_mk2 + lambda_mk3) * I_mk) / R_mjd

            fai_mj1 += (uplink_fai + downlink_fai)

            f_mjk = po[2][m][j][k]
            fai_mj2 += S_j * I_mk * lambda_mk2 * alpha_mk * (f_mjk ** 2)

        return fai_mj1 + fai_mj2

    def cal_Emj():

        def cal_fai_m0(m):
            fai_m0 = 0.0
            for k in range(K):
                lambda_mk3 = po[5][m][k]
                uplink_fai = (P_theta1 * beta_3 * lambda_mk3 * I_mk) / r0
                downlinke_fai = (P_theta2 * beta_4 * lambda_mk3 * I_mk) / r0
                execute_fai = I_mk * lambda_mk3 * alpha_mk * e0

                fai_m0 += (uplink_fai + downlinke_fai + execute_fai)

            return fai_m0

        total = 0.0
        for m in range(M):
            for j in range(J):
                x_mj = po[0][m][j]
                fai_mj = cal_Fai_mj(m, j)
                fai_m0 = cal_fai_m0(m)
                E_mj = (fai_mj + fai_m0)
                total += x_mj * E_mj

        return total

    def penalty():

        def time_constrain():
            def cal_Tmjk(m, k):
                lambda_mk2 = po[4][m][k]
                lambda_mk3 = po[5][m][k]
                total_time = 0.0
                for j in range(J):
                    x_mj = po[0][m][j]
                    R_mju, R_mjd = cal_Rmj(m, j)
                    f_mjk = po[2][m][j][k]
                    sbs_uplink = (beta_1 * (lambda_mk2 + lambda_mk3) * I_mk) / R_mju
                    sbs_exe = (lambda_mk2 * I_mk * alpha_mk) / f_mjk
                    sbs_downlink = (beta_2 * (lambda_mk2 + lambda_mk3) * I_mk) / R_mjd
                    sbs_time = (sbs_uplink + sbs_exe + sbs_downlink)

                    mbs_uplink = (beta_3 * lambda_mk3 * I_mk) / r0
                    mbs_exe = (lambda_mk3 * I_mk * alpha_mk) / f0
                    mbs_downlink = (beta_4 * lambda_mk3 * I_mk) / r0
                    mbs_time = (mbs_uplink + mbs_exe + mbs_downlink)

                    total_time += (x_mj * (sbs_time + mbs_time))
                return total_time

            T_diff = []
            for m in range(M):
                tm = 0
                for k in range(K):
                    lambda_mk1 = po[3][m][k]
                    f_mk = po[1][m][k]
                    T_mk = (I_mk * lambda_mk1 * alpha_mk) / f_mk
                    T_mkj = cal_Tmjk(m, k)
                    tm += max(T_mk, T_mkj)
                T_diff.append(max(0, tm - T_mstar))

            return (np.array(T_diff) ** 2).sum()

        def lambda_constrain():
            lambda_diff = []
            for m in range(M):
                for k in range(K):
                    lambda_mk1 = po[3][m][k]
                    lambda_mk2 = po[4][m][k]
                    lambda_mk3 = po[5][m][k]
                    diff = lambda_mk1 + lambda_mk2 + lambda_mk3 - 1
                    lambda_diff.append(diff)
            return (np.array(lambda_diff) ** 2).sum()

        def x_mj_constrain():
            x_sum_diff = []
            for m in range(M):
                sum = 0
                for j in range(J):
                    sum += po[0][m][j]
                diff = sum - 1
                x_sum_diff.append(diff)
            return (np.array(x_sum_diff) ** 2).sum()

        def sbs_constrain():
            cpu_diff = []
            mem_diff = []
            for j in range(J):
                cpu_cycles = 0
                mem = 0
                for m in range(M):
                    x_mj = po[0][m][j]
                    for k in range(K):
                        lambda_mk2 = po[4][m][k]
                        cpu_cycles += x_mj * I_mk * lambda_mk2 * alpha_mk
                        mem += x_mj * lambda_mk2 * I_mk * X_mk
                cpu_diff.append(max(0, cpu_cycles - A_1j))
                mem_diff.append(max(0, mem - G_1j))
            return (np.array(cpu_diff) ** 2).sum() + (np.array(mem_diff) ** 2).sum()

        def mbs_constrain():
            cpu_diff = 0.0
            mem_diff = 0.0
            mem = 0
            cpu = 0
            for m in range(M):
                for k in range(K):
                    lambda_mk3 = po[5][m][k]
                    cpu += I_mk * lambda_mk3 * alpha_mk
                    mem += I_mk * lambda_mk3 * X_mk

            cpu_diff = max(0, cpu - A_2j)
            mem_diff = max(0, mem - G_2j)
            return cpu_diff ** 2 + mem_diff ** 2

        def energy_constrain():
            def MDs_energy():
                e_diff = []
                for m in range(M):
                    e = 0
                    for k in range(K):
                        lambda_mk1 = po[3][m][k]
                        f_mk = po[1][m][k]
                        e += S_m * I_mk * lambda_mk1 * alpha_mk * (f_mk ** 2)
                    e_diff.append(max(0, e - E_m))
                return (np.array(e_diff) ** 2).sum()

            def SBSs_engry():
                e_diff = []
                for j in range(J):
                    e = 0
                    for m in range(M):
                        x_mj = po[0][m][j]
                        fai_mj = cal_Fai_mj(m, j)
                        e += x_mj * fai_mj
                    e_diff.append(max(0, e - E_j))
                return (np.array(e_diff) ** 2).sum()

            def MBS_engry():
                e_diff = 0.0
                e = 0
                for m in range(M):
                    for k in range(K):
                        lambda_mk3 = po[5][m][k]
                        uplink_e = (P_theta1 * beta_3 * lambda_mk3 * I_mk) / r0
                        downlink_e = (P_theta2 * beta_4 * lambda_mk3 * I_mk) / r0
                        exe_e = I_mk * lambda_mk3 * alpha_mk * e0
                        e += (uplink_e + downlink_e + exe_e)
                e_diff = max(0, e - E_0)
                return e_diff ** 2

            return MDs_energy() + SBSs_engry() + MBS_engry()

        def x_mu_constrain():
            x_mu_diff = []
            mu_min_diff = []
            mu_max_diff = []
            for j in range(J):
                sum = 0
                for m in range(M):
                    x_mj = po[0][m][j]
                    mu_mj = po[-1][m][j]
                    sum += x_mj * mu_mj
                    mu_min_diff.append(max(0, -mu_mj))
                    mu_max_diff.append(max(0, mu_mj - 1))
                x_mu_diff.append(sum - 1)
            return (np.array(x_mu_diff) ** 2).sum()

        def pt_constrain():
            pt_diff = []
            for m in range(M):
                pt = po[-2][m]
                pt_diff.append(max(0, pt - P_mt))
            return (np.array(pt_diff) ** 2).sum()

        def fm_constrain():
            fm_diff = []
            for m in range(M):
                fm = 0
                for k in range(K):
                    f_mk = po[1][m][k]
                    fm += f_mk
                fm_diff.append(max(0, fm - Fm))
            return (np.array(fm_diff) ** 2).sum()

        def fmj_constrain():
            fmj_diff = []
            for j in range(J):
                fmj = 0
                for m in range(M):
                    x_mj = po[0][m][j]
                    for k in range(K):
                        f_mjk = po[2][m][j][k]
                        fmj += f_mjk
                fmj_diff.append(max(0, fmj - Fj))
            return (np.array(fmj_diff) ** 2).sum()

        total_penalty = time_constrain() + \
                        lambda_constrain() + \
                        x_mj_constrain() + \
                        sbs_constrain() + \
                        mbs_constrain() + \
                        energy_constrain() + \
                        x_mu_constrain() + \
                        pt_constrain() + \
                        fm_constrain() + \
                        fmj_constrain()
        return total_penalty

    optim = cal_Em() + cal_Emj() + N * penalty()

    return optim


class Individual:
    def __init__(self, vector,D):
        self.vector = vector
        self.fitness_score = optim_score(vector)
        self.p_best = vector
        self.p_best_score = self.fitness_score
        self.N = N
        self.D = D
        self.velocity = np.zeros(self.D)

        self.exemplar_vector = None
        self.exemplar_score = None

    # def update(self):
        # self.fitness_score = optim_score(self.vector)
        # if self.fitness_score < self.p_best_score:
        #     self.p_best_score = self.fitness_score
        #     self.p_best = self.vector
        #     self.exemplar_vector =



In [ ]:
import numpy as np
# from papper_sbs_mbs.parameters import *
# from Individual import *
import copy


class PGL:
    def __init__(self,prob,c,c1,c2,max_iters,w_u,w_b,nums):
        self.prob = prob
        self.c = c
        self.c1 = c1
        self.c2 = c2
        self.max_iters = max_iters
        self.w_u = w_u
        self.w_b = w_b
        self.nums = nums

        self.segment = [M * J,M * K,M * K * J,M * K,M * K,M * K,M,M * J]
        for i in range(1,len(self.segment)):
            self.segment[i] += self.segment[i - 1]
        # print(self.segment)


        self.global_best = {
            "score": None,
            "vector": None
        }

        np.random.seed(3)
        self.D = M * (4 * K + 2 * J + J * K + 1)

        self.population = self.generate_populations()


    def list_flatten(self,list):
        v = np.zeros(0)
        # print(v)
        for each in list:
            temp = each.reshape(-1)
            v = np.append(v,temp)
        v = np.array(v).reshape(-1)
        # print(v.shape)
        return v

    def flatten_package(self,vector):
        start = 0
        # print(len(vector))
        x_mj = vector[0:self.segment[0]].reshape(M,J)

        f_mk = vector[self.segment[0]:self.segment[1]].reshape(M,K)
        f_mjk = vector[self.segment[1]:self.segment[2]].reshape(M,J,K)
        lambda_mk1 = vector[self.segment[2]:self.segment[3]].reshape(M,K)
        lambda_mk2 = vector[self.segment[3]:self.segment[4]].reshape(M, K)
        lambda_mk3 = vector[self.segment[4]:self.segment[5]].reshape(M, K)
        p_mt = vector[self.segment[5]:self.segment[6]].reshape(M)
        mu_mj = vector[self.segment[6]:self.segment[7]].reshape(M, J)
        v = [x_mj.reshape(M, J),
                  f_mk.reshape(M, K),
                  f_mjk.reshape(M, J, K),
                  lambda_mk1.reshape(M, K),
                  lambda_mk2.reshape(M, K),
                  lambda_mk3.reshape(M, K),
                  p_mt,
                  mu_mj.reshape(M, J)]
        return v


    def get_exeamplar(self,each):
        # e_vector = copy.deepcopy(each.vector)
        p_vector = copy.deepcopy(each.p_best)
        g_vector = copy.deepcopy(self.global_best["vector"])

        pd = self.list_flatten(p_vector)
        gd = self.list_flatten(g_vector)

        r1 = np.random.randn()
        r2 = np.random.randn()

        e_vector = ((self.c1 * r1 * pd) + (self.c2 * r2 * gd)) / (self.c1 * r1 + self.c2 * r2)
        return self.flatten_package(e_vector)





    def generate_populations(self):
        # pos = np.random.choice(1)
        populations = []

        def generate():
            temp_list = []
            for i in range(M * J):
                temp_list.append(np.long(np.random.choice(2)))
            x_mj = np.array(temp_list)
            temp_list.clear()

            for i in range(M * K):
                temp_list.append(np.random.randint(Fm))
            f_mk = np.array(temp_list)
            temp_list.clear()

            for i in range(M * K * J):
                temp_list.append(np.random.randint(Fj))
            f_mjk = np.array(temp_list)
            temp_list.clear()


            for i in range(M * K):
                temp_list.append(np.random.uniform(0, 1))
            lambda_mk1 = np.array(temp_list)
            temp_list.clear()

            for i in range(M * K):
                temp_list.append(np.random.uniform(0, 1-lambda_mk1[i]))
            lambda_mk2 = np.array(temp_list)
            temp_list.clear()


            lambda_mk3 = np.array(1 - lambda_mk1 - lambda_mk2)

            for i in range(M):
                temp_list.append(np.random.uniform(P_mt))
            p_mt = np.array(temp_list)
            temp_list.clear()

            for i in range(M * J):
                temp_list.append(np.random.uniform(0, 1))
            mu_mj = np.array(temp_list)
            temp_list.clear()

            vector = [x_mj.reshape(M,J),
                    f_mk.reshape(M,K),
                    f_mjk.reshape(M,J,K),
                    lambda_mk1.reshape(M,K),
                    lambda_mk2.reshape(M,K),
                    lambda_mk3.reshape(M,K),
                    p_mt,
                    mu_mj.reshape(M,J)]

            return vector

        for n in range(self.nums):
            individual = Individual(generate(),self.D)
            if self.global_best["score"] == None  or individual.fitness_score < self.global_best["score"]:
                self.global_best["score"] = individual.fitness_score
                self.global_best["vector"] = individual.vector
            populations.append(individual)

        for each in populations:
            exemplar = self.get_exeamplar(each)
            each.exemplar_vector = exemplar
            each.exemplar_score = optim_score(exemplar)


        return populations


    def optim(self):

        def crossover(o,other,each,d):
            if each.fitness_score < other.fitness_score:
                p_vector = copy.deepcopy(each.vector)
                g_vector = copy.deepcopy(self.global_best["vector"])
                pd = self.list_flatten(p_vector)
                gd = self.list_flatten(g_vector)

                rd = np.random.uniform(0,1)
                # print(len(o),len(pd),len(gd))
                o[d] = rd * pd[d] + (1 - rd) * gd[d]
            else:
                z = copy.deepcopy(other.vector)
                z_v = self.list_flatten(z)
                o[d] = z_v[d]

        def mutation(o,d):
            rd = np.random.uniform()
            if rd < self.prob:
                if d < M*J:
                    o[d] = np.random.randint(2)
                elif d < M*J + M*K:
                    o[d] = np.random.randint(0,Fm)
                elif d < M*J + M*K + M*J*K:
                    o[d] = np.random.randint(0,Fj)
                elif d < M*J + M*K + M*J*K + 3*M*K:
                    o[d] = np.random.uniform(0,1)
                elif d < M*J + M*K + M*J*K + 3*M*K + M:
                    o[d] = np.random.uniform(0,P_mt)
                else:
                    o[d] = np.random.uniform(0,1)
                    
                    
        def bound_value(v):
            vector = self.flatten_package(v)
            #x_mj bound
            for m in range(M):
                for j in range(J):
                    if vector[0][m][j] < 0.5:
                        vector[0][m][j] = 0
                    if vector[0][m][j] > 0.5:
                        vector[0][m][j] = 1
            #fm
            for m in range(M):
                for k in range(K):
                    if vector[1][m][k] < 0:
                        vector[1][m][k] = 0
                    if vector[1][m][k] > Fm:
                        vector[1][m][k] = Fm
            #fmj
            for m in range(M):
                for j in range(J):
                    for k in range(K):
                        if vector[2][m][j][k] < 0:
                            vector[2][m][j][k] = 0
                        if vector[2][m][j][k] > Fj:
                            vector[2][m][j][k] = Fj
            #lambda
            for m in range(M):
                for k in range(K):
                    if vector[3][m][k] < 0:
                        vector[3][m][k] = 0
                    if vector[3][m][k] > 1:
                        vector[3][m][k] = 1

                    if vector[4][m][k] < 0:
                        vector[4][m][k] = 0
                    if vector[4][m][k] > 1:
                        vector[4][m][k] = 1

                    if vector[5][m][k] < 0:
                        vector[5][m][k] = 0
                    if vector[5][m][k] > 1:
                        vector[5][m][k] = 1

            #pm
            for m in range(M):
                if vector[6][m] < 0:
                    vector[6][m] = 0
                if vector[6][m] > P_mt:
                    vector[6][m] = P_mt

            #mu
            for m in range(M):
                for j in range(J):
                    if vector[-1][m][j] < 0:
                        vector[-1][m][j] = 0
                    if vector[-1][m][j] > 1:
                        vector[-1][m][j] = 1
            return vector



        def update(each,candiate,w):
            # print(candiate)
            candiate_v = self.list_flatten(candiate)
            each_v = self.list_flatten(each.vector)
            rd = np.random.uniform(0,1)
            v = w * each.velocity + self.c*rd*(candiate_v - each_v)
            each_v += v
            for d in range(len(each_v)):
                if each_v[d] < 0:
                    each_v[d] = 0
                    v[d] = 0.1 * (-v[d])
            each.velocity = v

            each.vector = bound_value(each_v)
            each.velocity = self.list_flatten(bound_value(v))

#             print(each.vector)
            if each.fitness_score < each.p_best_score:
#                 print(each.vector[0])
                each.p_best_score = each.fitness_score
                each.p_best_vector = each.vector
                each.exemplar = self.get_exeamplar(each)
                each.exemplar_score = optim_score(each.exemplar)
            if each.fitness_score < self.global_best["score"]:
                self.global_best["score"] = each.fitness_score
                self.global_best["vector"] = each.vector
            return energy(each.vector)





        for iter in range(self.max_iters):
            w = self.w_u - (iter * (self.w_u - self.w_b))/self.max_iters
          
            for each in self.population:
                o_each = self.list_flatten(copy.deepcopy(each.vector))

                for d in range(self.D):
                    other = self.population[np.random.choice(self.nums)]
                    crossover(o_each,other,each,d)
                    mutation(o_each,d)

                o_vector = self.flatten_package(o_each)
                candidate = None
                if optim_score(o_vector) < each.exemplar_score:
                    candidate = o_vector
                else:
                    candidate = each.exemplar_vector

                update(each,candidate,w)

            best_e = energy(self.global_best["vector"])
            print("best_score: ",self.global_best["score"],"best_e: ",best_e)
            





if __name__ == '__main__':
    pgl = PGL(0.05,1.49618,0.5,0.5,500,0.95,0.4,100)
    pgl.optim()
#     prob,c,c1,c2,max_iters,w_u,w_b,nums
























